In [1]:
%matplotlib inline
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

In [2]:
orig_img_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data/data/images'
# img_width, img_height = 640., 480.
data_df = pd.read_csv(os.path.join('X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data', 'training_set_s3.csv'))

In [ ]:
def vertical_flip(image_array: ndarray):
    return cv2.flip( img, 0)

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return cv2.flip(img, 1)

def hv_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return cv2.flip(img, -1)

In [ ]:
im = os.path.join(orig_img_dir, data_df.image_name[3])
img = cv2.imread(im)
img_h_flip = horizontal_flip(img).astype(np.uint8) # noisy('gauss',img).astype(np.uint8)
img_v_flip = vertical_flip(img).astype(np.uint8)
img_hv_flip = hv_flip(img).astype(np.uint8)
plt.figure(0)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.figure(1)
plt.imshow(cv2.cvtColor(img_h_flip, cv2.COLOR_BGR2RGB))
plt.figure(2)
plt.imshow(cv2.cvtColor(img_v_flip, cv2.COLOR_BGR2RGB))
plt.figure(3)
plt.imshow(cv2.cvtColor(img_hv_flip, cv2.COLOR_BGR2RGB))

In [ ]:
df1 = pd.DataFrame({'image_name': ['aug_im'], 'x1': [1], 'x2': [2], 'y1': [3], 'y2' :[4]})
df2 = pd.DataFrame({'image_name': ['aug_im'], 'x1': [1], 'x2': [2], 'y1': [3], 'y2' :[4]})
df1 = df1.append(df2, ignore_index=True)
df1.head()

In [ ]:
data_df = data_df.sample(frac=1).reset_index(drop=True)
val_df = data_df[22000:]
res_df = data_df[:22000]
val_df.to_csv()

In [ ]:

# Horizontal Flip Loop
for i, im in enumerate(data_df.image_name):
    img = cv2.imread(os.path.join(orig_img_dir, im))
    x1 = data_df.iloc[i].x1
    x2 = data_df.iloc[i].x2
    y1 = data_df.iloc[i].y1
    y2 = data_df.iloc[i].y2
    aug_im = '[h]' + im
    img_h = horizontal_flip(img).astype(np.uint8)
    img_h_lab = pd.DataFrame({'image_name': [aug_im], 'x1': [img_width - x2], 'x2': [img_width - x1], 'y1': [y1], 'y2' :[y2]})
    cv2.imwrite(os.path.join(orig_img_dir, aug_im), img_h)
    res_df = res_df.append(img_h_lab, ignore_index = True)
    

In [ ]:
res_df.head()
# res_df.tail()

In [ ]:
# Vertical Flip Loop
for i, im in enumerate(data_df.image_name):
    img = cv2.imread(os.path.join(orig_img_dir, im))
    x1 = data_df.iloc[i].x1
    x2 = data_df.iloc[i].x2
    y1 = data_df.iloc[i].y1
    y2 = data_df.iloc[i].y2
    aug_im = '[v]' + im
    img_v = vertical_flip(img).astype(np.uint8)
    img_v_lab = pd.DataFrame({'image_name': [aug_im], 'x1': [x1], 'x2': [x2], 'y1':[img_height - y2], 'y2': [img_height - y1] })
    cv2.imwrite(os.path.join(orig_img_dir, aug_im), img_v)
    res_df = res_df.append(img_v_lab, ignore_index = True)

In [ ]:
# HorizontalVertical Flip Loop
for i, im in enumerate(data_df.image_name):
    img = cv2.imread(os.path.join(orig_img_dir, im))
    x1 = data_df.iloc[i].x1
    x2 = data_df.iloc[i].x2
    y1 = data_df.iloc[i].y1
    y2 = data_df.iloc[i].y2
    aug_im = '[h_v]' + im
    img_h_v = hv_flip(img).astype(np.uint8)
    img_h_v_lab = pd.DataFrame({'image_name': [aug_im], 'x1': [img_width-x2], 'x2': [img_width-x1], 'y1':[img_height - y2], 'y2': [img_height - y1]})
    cv2.imwrite(os.path.join(orig_img_dir, aug_im), img_h_v)
    res_df = res_df.append(img_h_v_lab, ignore_index = True)

In [ ]:
#Gaussian Noise Loop
for i, im in enumerate(data_df.image_name):
    img = cv2.imread(os.path.join(orig_img_dir, im))
    x1 = data_df.iloc[i].x1
    x2 = data_df.iloc[i].x2
    y1 = data_df.iloc[i].y1
    y2 = data_df.iloc[i].y2
    aug_im = '[g]' + im
    img_g = noisy("gauss", img).astype(np.uint8)
    img_g_lab = pd.DataFrame({'image_name': [aug_im], 'x1': [x1], 'x2': [x2], 'y1':[y1], 'y2': [y2]})
    cv2.imwrite(os.path.join(orig_img_dir, aug_im), img_g)
    res_df = res_df.append(img_g_lab, ignore_index = True)

In [ ]:
print(res_df.image_name.values.shape)
res_df.head()

In [ ]:
augment_path = 'X:\Flipkart GRID - Object Localization (Medium Complexity Round)\\augument_data'
res_df.to_csv(os.path.join(augment_path, 'augumented_data.csv'), index=False)

In [ ]:
# data_df = data_df.sample(frac=1).reset_index(drop=True)
res_df = res_df.sample(frac=1).reset_index(drop=True)

In [ ]:
res_df = pd.read_csv(os.path.join(augment_path, 'augumented_data.csv'))

In [ ]:
res_df = res_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_df = res_df.iloc[:68000]
val_df = res_df.iloc[68000:]

In [ ]:
train_df.to_csv(os.path.join(augment_path, 'augmented_train_data.csv'), index=False)
val_df.to_csv(os.path.join(augment_path, 'augmented_val_data.csv'), index=False)

In [ ]:
res_df.head()

In [ ]:
res_df.tail()

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
val_df.head()

In [ ]:
val_df.tail()

In [4]:
augment_path = 'X:\Flipkart GRID - Object Localization (Medium Complexity Round)\\augument_data'
train_df = pd.read_csv(os.path.join(augment_path, 'augmented_train_data.csv'))

In [5]:
# print(len(train_df.image_name.values))
img_width, img_height = (299, 299)

In [6]:
# for i in range(0,len(train_df.image_name.values), 1000):
#     j = i
#     l = []
#     for j in range(i, i+1000):
#         img = (cv2.imread(os.path.join(augment_path, 'training_images', train_df.iloc[j].image_name)) #/ 255.).astype(np.float32)
#         img = cv2.resize(img, (299, 299))
#         img = cv2.resize(img, (224, 224))
#         img = (img / 255.).astype(np.float32)
#         l.append(np.expand_dims(img, axis = 0))
#     l = np.array(l)
#     train_batch = np.vstack(l)
#     #     with open('x_train.pickle', 'wb') as handle:
#     #         pickle.dump(train_batch, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     np.savez_compressed(os.path.join(augment_path, 'training_images_batches', str(i//1000)), train_batch)
    

In [8]:
for i, im in enumerate(tqdm(train_df.image_name.values)):
    img = cv2.imread(os.path.join(augment_path, 'training_images', im))
    img = cv2.resize(img, (img_width, img_height))
#     img = (img / 255.).astype(np.float32)
    cv2.imwrite(os.path.join(augment_path, 'resized_training_images', im), img)
    

100%|████████████████████████████████████████████████████████████████████████████| 68000/68000 [14:25<00:00, 78.54it/s]


In [ ]:
for i in range(0, len(train_df.image_name.values):
    print(i)
    j = i
    l = []
    for j in range(i, i+500):
        img = cv2.imread(os.path.join(augment_path, 'training_images', train_df.iloc[j].image_name))
        img = cv2.resize(img, (img_width, img_height))
#         img = (img / 255.).astype(np.float32)
#         l.append(np.expand_dims(img, axis = 0))
        cv2.imwrite(os.path.j)
    l = np.array(l)
    train_batch = np.vstack(l)
    np.save(os.path.join(augment_path, 'resized_training_images', str(i//500)), train_batch)

In [ ]:
im = train_df.iloc[j].image_name
print(i9m)

In [ ]:
for i, im in enumerate(train_df.image_name):
    if im[:5] == '[h_v]':
        idx = i
        break

In [ ]:
print(idx)

In [ ]:
n = np.load(os.path.join(augment_path, 'training_images_batches', '0.npy'))

In [ ]:
print(n.shape)

In [ ]:
img = n[idx]

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
x1 = int(train_df.x1.iloc[idx]/2.1404)
x2 = int(train_df.x2.iloc[idx]/2.1404)
y1 = int(train_df.y1.iloc[idx]/1.61)
y2 = int(train_df.y2.iloc[idx]/1.61) #
cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
for i, im in enumerate(train_df.image_name.iloc[0:10]):
    print(im)
    plt.figure(i)
    img = cv2.imread(os.path.join(augment_path, 'training_images', im))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Test NPY

In [ ]:
base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data'
test_df = pd.read_csv(os.path.join(base_dir, 'test.csv'))

In [ ]:
img_width, img_height = (299, 299)
for i in range(0, len(test_df.image_name.values), 1000):
    print(i)
    j = i
    l = []
    for j in range(i, i+1000):
        if(j >= 12815):
            break
        img = cv2.imread(os.path.join(base_dir, 'data', 'images', test_df.iloc[j].image_name))
        img = cv2.resize(img, (img_width, img_height))
        img = (img / 255.).astype(np.float32)
        l.append(np.expand_dims(img, axis = 0))
    l = np.array(l)
    test_batch = np.vstack(l)
    np.save(os.path.join(augment_path, 'test_images_batches', str(i//1000)), test_batch)
    